In [ ]:
#################IMPORTS
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import model_from_json
import copy
import cv2, time, math, json
import numpy as np
import solver
#################IMPORTS





#################NETWORK SETUP
input_shape = (28, 28, 1)
num_classes = 9
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.load_weights("digitRecognition.h5") 
#################NETWORK SETUP





video = cv2.VideoCapture(0)


while True:

    #########PREPROCESSING
    check, frame = video.read()
    og = frame
    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    frame = cv2.GaussianBlur(frame,(5,5),0)
    frame = cv2.adaptiveThreshold(frame, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 5)
    #########PREPROCESSING











    ###########SELECT GRID

    blank1, contours, blank2 = cv2.findContours(frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    maxA = -1
    maxC = None
    secA = -1
    secC = None

    for f in contours:
        if cv2.contourArea(f) > maxA:
            maxA = cv2.contourArea(f)
            maxC = f
        elif cv2.contourArea(f) > secA:
            secA = cv2.contourArea(f)
            board = f
    # if board is None:
    #     continue

    epsilon = 0.025*cv2.arcLength(board, True)
    poly = cv2.approxPolyDP(board, epsilon, True)


    if len(poly) > 3:

        topleft =       min(board, key=lambda x: x[0,0]+x[0,1])
        bottomright =   max(board, key=lambda x: x[0,0]+x[0,1])
        topright =      max(board, key=lambda x: x[0,0]-x[0,1])
        bottomleft =    min(board, key=lambda x: x[0,0]-x[0,1])
        corners = (topleft, topright, bottomleft, bottomright)
        corners = np.array(corners) 
    rect = cv2.minAreaRect(corners)
    width = int(rect[1][0])
    height = int(rect[1][1])

    src_pts = np.float32([corners[0],corners[1],corners[2],corners[3]])
    dst_pts = np.float32([[0,0],[width,0],[0,height],[width,height]])

    per = cv2.getPerspectiveTransform(src_pts, dst_pts)

    warp = cv2.warpPerspective(frame, per, (width, height))
    color_warp = cv2.warpPerspective(og, per, (width, height))

    ###########SELECT GRID

















    sudoku_border = 4
    border = 2
    x = width/9
    y = height/9
    prid = [None]*81
    grid = [0]*81
    contains = [0]*81
    for i in range(9):
        for j in range(9):



            top     = int(round(y*i+border))
            left    = int(round(x*j+border))
            right   = int(round(x*(j+1)-border))
            bottom  = int(round(y*(i+1)-border))
            if i == 0:
                top+=sudoku_border
            if i == 8:
                bottom-=sudoku_border
            if j == 0:
                left+=sudoku_border 
            if j == 8:
                right-=sudoku_border

            corners = [
                [[left,  top]],
                [[right, top]],
                [[left,  bottom]],
                [[right, bottom]]
                ]

    ############CROP FROM POINT######################    

            cnt = np.array([
                corners[0],
                corners[1],
                corners[2],
                corners[3]
                ])

            rect = cv2.minAreaRect(cnt)
            center, size, theta = rect


            if theta < -45:
                theta += 90

            rect = (center, size, theta)

            box = cv2.boxPoints(rect)
            box = np.int0(box)




            width = int(rect[1][0])
            height = int(rect[1][1])

            src_pts = np.float32([corners[0],corners[1],corners[2],corners[3]])
            dst_pts = np.float32([[0,0],[width,0],[0,height],[width,height]])


            M = cv2.getPerspectiveTransform(src_pts, dst_pts)


            warped = cv2.warpPerspective(warp, M, (width, height))



    ############CROP FROM POINT######################






    ###############FIND EMPTY BOXES##################
            prid[9*i+j] = cv2.resize(warped, (28,28), interpolation = cv2.INTER_CUBIC)
            center_width = prid[9*i+j].shape[1] // 2
            center_height = prid[9*i+j].shape[0] // 2
            x_start = center_height // 2
            x_end = center_height // 2 + center_height
            y_start = center_width // 2
            y_end = center_width // 2 + center_width
            center_region = prid[9*i+j][x_start:x_end, y_start:y_end]
            if center_region.sum() >= center_width * center_height * 255 - 255:
                contains[9*i+j] = 11
    ###############FIND EMPTY BOXES##################




            for a in range(0,28):
                prid[9*i+j][0,a] = 255
                prid[9*i+j][27,a] = 255
                prid[9*i+j][a,0] = 255 
                prid[9*i+j][27,a] = 255







    ##############CROP NUMBER ITSELF AND RESIZE######################
            if contains[9*i+j] == 0:

                blank1, contours, blank2 = cv2.findContours(warped, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                if contours:
                    conts = sorted(contours, key=cv2.contourArea, reverse=True)

                    cnt = conts[0]####################


                    minarea = x*y*0.03
                    if cv2.contourArea(cnt) > minarea:


                        rect = cv2.minAreaRect(cnt)
                        box = cv2.boxPoints(rect)
                        box = np.int0(box)
                        minx = max(min(box, key=lambda g: g[0])[0], 0)
                        miny = max(min(box, key=lambda g: g[1])[1], 0)
                        maxx = min(max(box, key=lambda g: g[0])[0], int(x))
                        maxy = min(max(box, key=lambda g: g[1])[1], int(y))

                        warped = warped[miny:maxy, minx:maxx]

                        prid[9*i+j] = cv2.resize(warped, (28,28), interpolation = cv2.INTER_CUBIC)

    ##############CROP NUMBER ITSELF AND RESIZE######################







    #############PREP AND PREDICT#####################
                        img = prid[9*i+j] 
                        img = img.astype(np.uint8)    
                        img = img.reshape(-1, 28, 28, 1)
                        img = img.astype('float32')
                        img /= 255  
                        prediction = model.predict([img])
                        if 9*i+j == 72:
                            print(prediction)
                        grid[9*i+j] = np.argmax(prediction[0]) +1
    #############PREP AND PREDICT#####################










    ###########SOLVE#####################                    
    grid = np.reshape(grid,(9,9))
    blank_grid = copy.deepcopy(grid)
    solver.solve_sudoku(grid)    
    ###########SOLVE##################### 









    #######PRINT TEXT ON BOARD###############
    if grid is not None:
        SIZE = 9
        width = color_warp.shape[1] // 9
        height = color_warp.shape[0] // 9
        for i in range(SIZE):
            for j in range(SIZE):
                if blank_grid[i][j] == 0: 
                    text = str(grid[i][j])
                    off_set_x = width // 15
                    off_set_y = height // 15
                    font = cv2.FONT_HERSHEY_SIMPLEX
                    (text_height, text_width), baseLine = cv2.getTextSize(text, font, fontScale=1, thickness=1)
                    marginX = math.floor(width / 7)
                    marginY = math.floor(height / 7)
                    font_scale = 0.6 * min(width, height) / max(text_height, text_width)
                    text_height *= font_scale
                    text_width *= font_scale
                    bottom_left_corner_x = width*j + math.floor((width - text_width) / 2) + off_set_x
                    bottom_left_corner_y = height*(i+1) - math.floor((height - text_height) / 2) + off_set_y
                    color_warp = cv2.putText(color_warp, text, (bottom_left_corner_x, bottom_left_corner_y),font, font_scale, (255,0,0), thickness=3, lineType=cv2.LINE_AA)    
    #######PRINT TEXT ON BOARD###############





    ########SHOW#############

    final = cv2.warpPerspective(color_warp, per, (og.shape[1], og.shape[0])
                                        , flags=cv2.WARP_INVERSE_MAP)
    final = np.where(final.sum(axis=-1,keepdims=True)!=0, final, og)        

    cv2.imshow('solved_grid',final)

    key = cv2.waitKey(1)
    if key == ord('x'):
        break


    #######SHOW#############













video.release()
cv2.destroyAllWindows
